In [1]:
import pandas as pd
from sqlalchemy import create_engine
# In GitBash:
# $ pip install geocoder
import geocoder
import gmaps
import numpy as np
from includes import (gmaps_api_key)
%matplotlib inline


In [2]:
#create function for geo coding addresses
def geomapper(address):
    g = geocoder.google(address, key=gmaps_api_key)
    lat = g.json['lat']
    long = g.json['lng']
    return (f'{lat},{long}')

In [3]:
#Create function called locateAddresses that adds latitude and longitude columns for geomapping
#Assumes df have column "Address"

def locateAddresses(df):
    df["geo"] = 0
    df["geo"]= df.Address.apply(lambda x: geomapper(x))
    
    #split geo column to assign values in latitude and longitude 
    df[['Lat', 'Lng']] = df.geo.str.split(',', expand = True)
    #df = df.drop(columns=['geo'])
    del df['geo']
    return

In [4]:
#File 1: Events: SD_Events.csv

In [5]:
#Process File 1
SD_events_file = "Resources/SD_Events.csv"
SD_events_df = pd.read_csv(SD_events_file)
SD_events_df.head()

,Event,Description,Neighborhood,Date,Cost
0,15th Annual Martin Luther King Jr Day Parade,(Not Provided),Downtown,1/20/2019,$$
1,18th Annual St Patricks Day Parade Festival,(Not Provided),Balboa Park,3/16/2019,$
2,3rd Annual San Diego Bird Festival,(Not Provided),Mission Bay,5/12/2019,$$
3,4th Annual International Mariachi Festival,(Not Provided),South Bay,3/10/2019,$$
4,4th Annual Small Image Show,"Art Exhibition Featuring Works 10x10"" and Smaller",Downtown,4/21/2019,$$


In [6]:
#Convert '$$' in Cost column to numeric
# get the length of the string of column in a dataframe
#http://www.datasciencemadesimple.com/get-string-length-column-dataframe-python-pandas/
#df['Quarters_length'] = df['Quarters'].apply(len)

SD_events_df.Cost = SD_events_df.Cost.apply(len)
SD_events_df.head()

,Event,Description,Neighborhood,Date,Cost
0,15th Annual Martin Luther King Jr Day Parade,(Not Provided),Downtown,1/20/2019,2
1,18th Annual St Patricks Day Parade Festival,(Not Provided),Balboa Park,3/16/2019,1
2,3rd Annual San Diego Bird Festival,(Not Provided),Mission Bay,5/12/2019,2
3,4th Annual International Mariachi Festival,(Not Provided),South Bay,3/10/2019,2
4,4th Annual Small Image Show,"Art Exhibition Featuring Works 10x10"" and Smaller",Downtown,4/21/2019,2


In [7]:
#Add "Category" column with "Food and Drink in San Diego" value
SD_events_df["Category"] = "Events"
SD_events_df.head()

,Event,Description,Neighborhood,Date,Cost,Category
0,15th Annual Martin Luther King Jr Day Parade,(Not Provided),Downtown,1/20/2019,2,Events
1,18th Annual St Patricks Day Parade Festival,(Not Provided),Balboa Park,3/16/2019,1,Events
2,3rd Annual San Diego Bird Festival,(Not Provided),Mission Bay,5/12/2019,2,Events
3,4th Annual International Mariachi Festival,(Not Provided),South Bay,3/10/2019,2,Events
4,4th Annual Small Image Show,"Art Exhibition Featuring Works 10x10"" and Smaller",Downtown,4/21/2019,2,Events


In [8]:
#Find the datatypes
SD_events_df.dtypes

Event           object
Description     object
Neighborhood    object
Date            object
Cost             int64
Category        object
dtype: object

In [9]:
#Replace "Ongoing" value in Date column with blank to be able to change datatype of Date column 
#into a date type to be able to search and sort datafield by date
#Keepin in mind:  blank dates = NaT = "Ongoing"
SD_events_df = SD_events_df.replace({'Date':['Ongoing']},'')
SD_events_df['Date'] = pd.to_datetime(SD_events_df['Date'])

#Rename column "Event" to "Name"
SD_events_df = SD_events_df.rename(columns={"Event":"Name"})
SD_events_df.head()

,Name,Description,Neighborhood,Date,Cost,Category
0,15th Annual Martin Luther King Jr Day Parade,(Not Provided),Downtown,2019-01-20,2,Events
1,18th Annual St Patricks Day Parade Festival,(Not Provided),Balboa Park,2019-03-16,1,Events
2,3rd Annual San Diego Bird Festival,(Not Provided),Mission Bay,2019-05-12,2,Events
3,4th Annual International Mariachi Festival,(Not Provided),South Bay,2019-03-10,2,Events
4,4th Annual Small Image Show,"Art Exhibition Featuring Works 10x10"" and Smaller",Downtown,2019-04-21,2,Events


In [10]:
#Clean File 1
#Sort Dataframe by name
SD_events_df = SD_events_df.sort_values("Name")
SD_events_df.head()

,Name,Description,Neighborhood,Date,Cost,Category
0,15th Annual Martin Luther King Jr Day Parade,(Not Provided),Downtown,2019-01-20,2,Events
1,18th Annual St Patricks Day Parade Festival,(Not Provided),Balboa Park,2019-03-16,1,Events
2,3rd Annual San Diego Bird Festival,(Not Provided),Mission Bay,2019-05-12,2,Events
3,4th Annual International Mariachi Festival,(Not Provided),South Bay,2019-03-10,2,Events
4,4th Annual Small Image Show,"Art Exhibition Featuring Works 10x10"" and Smaller",Downtown,2019-04-21,2,Events


In [11]:
#File 2: Attractions: TripAdvisor.csv

In [12]:
#Process File 2
trip_advisor_file = "Resources/TripAdvisor.csv"
trip_advisor_df = pd.read_csv(trip_advisor_file)
trip_advisor_df.head()

,Name,Address,Category,# of Reviews,Cost
0,SeaWorld San Diego,"500 Sea World Drive, San Diego, CA 92109-7904",Things to do,9485,$$$
1,Mt. Soledad National Veterans Memorial,"6905 La Jolla Scenic Drive South, La Jolla, Sa...",Things to do,2240,$
2,Pacific Surf School,"4121 Mission Blvd, San Diego, CA 92109-5028",Boat Tours Water Sports,1156,$$
3,San Diego Fly Rides,"7444 Girard Ave, La Jolla, San Diego, CA 92037...",Tours,1113,$$
4,Belmont Park,"3146 Mission Blvd, San Diego, CA 92109-7767",Things to do,764,$


In [13]:
#Convert '$$' in Cost column to numeric
# get the length of the string of column in a dataframe
#http://www.datasciencemadesimple.com/get-string-length-column-dataframe-python-pandas/
#df['Quarters_length'] = df['Quarters'].apply(len)

trip_advisor_df.Cost = trip_advisor_df.Cost.apply(len)
trip_advisor_df.head()

,Name,Address,Category,# of Reviews,Cost
0,SeaWorld San Diego,"500 Sea World Drive, San Diego, CA 92109-7904",Things to do,9485,3
1,Mt. Soledad National Veterans Memorial,"6905 La Jolla Scenic Drive South, La Jolla, Sa...",Things to do,2240,1
2,Pacific Surf School,"4121 Mission Blvd, San Diego, CA 92109-5028",Boat Tours Water Sports,1156,2
3,San Diego Fly Rides,"7444 Girard Ave, La Jolla, San Diego, CA 92037...",Tours,1113,2
4,Belmont Park,"3146 Mission Blvd, San Diego, CA 92109-7767",Things to do,764,1


In [14]:
#Replace "# of Reviews" column to "Reviews"
trip_advisor_df = trip_advisor_df.rename(columns={"# of Reviews":"Reviews"})
#Convert "Reviews" datatype to int 
trip_advisor_df['Reviews'] = trip_advisor_df['Reviews'].astype('int')
trip_advisor_df.head()

,Name,Address,Category,Reviews,Cost
0,SeaWorld San Diego,"500 Sea World Drive, San Diego, CA 92109-7904",Things to do,9485,3
1,Mt. Soledad National Veterans Memorial,"6905 La Jolla Scenic Drive South, La Jolla, Sa...",Things to do,2240,1
2,Pacific Surf School,"4121 Mission Blvd, San Diego, CA 92109-5028",Boat Tours Water Sports,1156,2
3,San Diego Fly Rides,"7444 Girard Ave, La Jolla, San Diego, CA 92037...",Tours,1113,2
4,Belmont Park,"3146 Mission Blvd, San Diego, CA 92109-7767",Things to do,764,1


In [15]:
#get a list of the Category
category_list = trip_advisor_df["Category"].unique()
category_list = pd.DataFrame(category_list)
category_list.columns = ["Category List"]

#Sort Dataframe by Category List
category_list = category_list.sort_values("Category List")
category_list

,Category List
1,Boat Tours Water Sports
4,Classes Workshops
10,Concerts Shows
3,Food Drink
5,Fun Games
6,Nightlife
7,Outdoor Activities
9,Shopping
8,Spas Wellness
0,Things to do


In [16]:
trip_advisor_df = trip_advisor_df.replace({'Category':['Boat Tours  Water Sports']},\
                                                'Boat Tours & Water Sports')

trip_advisor_df = trip_advisor_df.replace({'Category':['Classes  Workshops']},\
                                                'Classes & Workshops')

trip_advisor_df = trip_advisor_df.replace({'Category':['Concerts  Shows']},\
                                                'Concerts & Shows')

trip_advisor_df = trip_advisor_df.replace({'Category':['Food  Drink']},\
                                                'Food & Drink')

trip_advisor_df = trip_advisor_df.replace({'Category':['Fun  Games']},\
                                                'Fun & Games')

trip_advisor_df = trip_advisor_df.replace({'Category':['Spas  Wellness']},\
                                                'Spas & Wellness')

trip_advisor_df.head()

,Name,Address,Category,Reviews,Cost
0,SeaWorld San Diego,"500 Sea World Drive, San Diego, CA 92109-7904",Things to do,9485,3
1,Mt. Soledad National Veterans Memorial,"6905 La Jolla Scenic Drive South, La Jolla, Sa...",Things to do,2240,1
2,Pacific Surf School,"4121 Mission Blvd, San Diego, CA 92109-5028",Boat Tours & Water Sports,1156,2
3,San Diego Fly Rides,"7444 Girard Ave, La Jolla, San Diego, CA 92037...",Tours,1113,2
4,Belmont Park,"3146 Mission Blvd, San Diego, CA 92109-7767",Things to do,764,1


In [17]:
#get new list of the Category to see changes
category_list = trip_advisor_df["Category"].unique()
category_list = pd.DataFrame(category_list)
category_list.columns = ["Category List"]

#Sort Dataframe by Category List
category_list = category_list.sort_values("Category List")
category_list

,Category List
1,Boat Tours & Water Sports
4,Classes & Workshops
10,Concerts & Shows
3,Food & Drink
5,Fun & Games
6,Nightlife
7,Outdoor Activities
9,Shopping
8,Spas & Wellness
0,Things to do


In [18]:
#Clean File 2
#Sort Dataframe by name
trip_advisor_df = trip_advisor_df.sort_values("Name")
trip_advisor_df.head()

,Name,Address,Category,Reviews,Cost
58,Action Sport Rentals,"3981 Mission Blvd, San Diego, CA 92109-6958",Boat Tours & Water Sports,18,2
79,Adventure Water Sports,"1710 W Mission Bay Dr Dana Hotel, San Diego, C...",Boat Tours & Water Sports,7,2
130,Adventure Water Sports,"2211 Pacific Beach Dr, San Diego, CA 92109-5626",Boat Tours & Water Sports,1,2
127,Affordable Jet Ski Tours,"4255 Mission Bay Dr, San Diego, CA 92109-5782",Boat Tours & Water Sports,1,2
77,Always Summer Surf School,"4904 Crystal Dr, San Diego, CA 92109-2062",Boat Tours & Water Sports,8,2


In [19]:
#File 3: Restaurants: Yelp_restaurants.csv

In [20]:
#Process File 3
yelp_file = "Resources/Yelp_restaurants.csv"
yelp_df = pd.read_csv(yelp_file)
yelp_df.head()

,Name,Address,Category,# of Reviews,Cost
0,Daybreak Island Grill,2296 Bacon St,Breakfast & Brunch,5111,$$
1,Bravo's Mexican Bistro & Cantina,5001 Newport Ave,Mexican,3054,$$
2,Caveman Pizza,1404 Vacation Rd,Pizza,2594,$$
3,Barefoot Bar & Grill,1404 Vacation Rd,American (New),1526,$$$
4,Saska's,3768 Mission Blvd,Steakhouses,1501,$$$$


In [21]:
#Convert '$$' in Cost column to numeric
# get the length of the string of column in a dataframe
#http://www.datasciencemadesimple.com/get-string-length-column-dataframe-python-pandas/
#df['Quarters_length'] = df['Quarters'].apply(len)

yelp_df.Cost = yelp_df.Cost.apply(len)
yelp_df.head()

,Name,Address,Category,# of Reviews,Cost
0,Daybreak Island Grill,2296 Bacon St,Breakfast & Brunch,5111,2
1,Bravo's Mexican Bistro & Cantina,5001 Newport Ave,Mexican,3054,2
2,Caveman Pizza,1404 Vacation Rd,Pizza,2594,2
3,Barefoot Bar & Grill,1404 Vacation Rd,American (New),1526,3
4,Saska's,3768 Mission Blvd,Steakhouses,1501,4


In [22]:
#Rename "# of Reviews" column to "Reviews"
yelp_df = yelp_df.rename(columns={"# of Reviews":"Reviews"})
#Convert "Reviews" datatype to int 
yelp_df['Reviews'] = yelp_df['Reviews'].astype('int')
yelp_df.head()

,Name,Address,Category,Reviews,Cost
0,Daybreak Island Grill,2296 Bacon St,Breakfast & Brunch,5111,2
1,Bravo's Mexican Bistro & Cantina,5001 Newport Ave,Mexican,3054,2
2,Caveman Pizza,1404 Vacation Rd,Pizza,2594,2
3,Barefoot Bar & Grill,1404 Vacation Rd,American (New),1526,3
4,Saska's,3768 Mission Blvd,Steakhouses,1501,4


In [23]:
#Rename "Category" column to "Description" column
yelp_df = yelp_df.rename(columns = {"Category": "Description"})

#Add "Category" column with "Food and Drink in San Diego" value
yelp_df["Category"] = "Food & Drink"
yelp_df.head()

,Name,Address,Description,Reviews,Cost,Category
0,Daybreak Island Grill,2296 Bacon St,Breakfast & Brunch,5111,2,Food & Drink
1,Bravo's Mexican Bistro & Cantina,5001 Newport Ave,Mexican,3054,2,Food & Drink
2,Caveman Pizza,1404 Vacation Rd,Pizza,2594,2,Food & Drink
3,Barefoot Bar & Grill,1404 Vacation Rd,American (New),1526,3,Food & Drink
4,Saska's,3768 Mission Blvd,Steakhouses,1501,4,Food & Drink


In [24]:
#Clean File 3 
yelp_df = yelp_df[["Name", "Description", "Address", "Category", "Reviews", "Cost"]]

#Sort Dataframe by name
yelp_df = yelp_df.sort_values("Name")
yelp_df.head()

,Name,Description,Address,Category,Reviews,Cost
37,Arslan's Gyros,Greek,3861 Mission Blvd,Food & Drink,322,2
91,BBQ House Bar & Grill,Bars,5025 Newport Ave,Food & Drink,23,2
98,BO-beau kitchen + bar,French,4996 W Point Loma Blvd,Food & Drink,6,4
86,Bao Beach,Asian Fusion,3735 Mission Blvd,Food & Drink,36,2
3,Barefoot Bar & Grill,American (New),1404 Vacation Rd,Food & Drink,1526,3


In [25]:
#Create database connection
connection_string = "root:Password@localhost/trip_planner_db?charset=utf8"
engine = create_engine(f'mysql://{connection_string}')

In [26]:
# Confirm tables
engine.table_names()

['attractions', 'events', 'restaurants']

In [27]:
#Delete all rows from the table and resets auto increment value for id to 1
engine.execute("TRUNCATE TABLE events;")
engine.execute("TRUNCATE TABLE attractions;")
engine.execute("TRUNCATE TABLE restaurants;")

In [28]:
#Load DataFrames into database
SD_events_df.to_sql(name='events', con=engine, if_exists='append', index=False)
trip_advisor_df.to_sql(name='attractions', con=engine, if_exists='append', index=False)
yelp_df.to_sql(name='restaurants', con=engine, if_exists='append', index=False)

In [29]:
#Test Queries

In [30]:
#Select from all tables all events, attractions and restaurants with cost range of 2 and 3.
expensive_activities_df = pd.read_sql_query('select * from (select * from events union select * from attractions union select * from restaurants) as a \
where a.Cost = 4 AND a.Address IS NOT NULL order by a.Category;', con=engine)
locateAddresses(expensive_activities_df)
expensive_activities_df

,id,Name,Address,Description,Category,Neighborhood,Reviews,Date,Cost,Lat,Lng
0,26,Draft,3105 Ocean Front Walk,American (Traditional),Food & Drink,None,842,None,4,32.7699845,-117.2526347
1,97,Tidal,1404 Vacation Rd,Seafood,Food & Drink,None,302,None,4,32.7741456,-117.2404383
2,67,Red Marlin Restaurant,1441 Quivira Rd,Seafood,Food & Drink,None,263,None,4,32.764958,-117.240322
3,48,Nautilus Pavilion,500 Sea World Dr,American (Traditional),Food & Drink,None,47,None,4,32.7641067,-117.2262649
4,3,BO-beau kitchen + bar,4996 W Point Loma Blvd,French,Food & Drink,None,6,None,4,32.753055,-117.2451753
5,91,The Firefly Restaurant & Bar,1710 W Mission Bay Dr,American (New),Food & Drink,None,328,None,4,32.7665996,-117.2373163
6,76,Saska's,3768 Mission Blvd,Steakhouses,Food & Drink,None,1501,None,4,32.7839428,-117.2529375
7,41,Fanuel Beach Park,"4000 Fanuel St, San Diego, CA 92109",None,Things to do,None,10,None,4,32.7913569,-117.2443928
8,26,Capehart Dog Park,"4754 Soledad Mtn Road, San Diego, CA 92109",None,Things to do,None,3,None,4,32.8076602,-117.2257623
9,79,Ocean Beach Municipal Pier,"1950 Abbott St, San Diego, CA 92107-2804",None,Things to do,None,34,None,4,32.7488045,-117.2522136


In [31]:
# Configure gmaps with API key
gmaps.configure(api_key=gmaps_api_key)

# Store 'Lat' and 'Lng' into  locations 
locations = expensive_activities_df[["Lat", "Lng"]].astype(float)
print(locations)
reviews = expensive_activities_df["Reviews"].astype(float)
print(reviews)

         Lat         Lng
0  32.769984 -117.252635
1  32.774146 -117.240438
2  32.764958 -117.240322
3  32.764107 -117.226265
4  32.753055 -117.245175
5  32.766600 -117.237316
6  32.783943 -117.252938
7  32.791357 -117.244393
8  32.807660 -117.225762
9  32.748804 -117.252214
0     842.0
1     302.0
2     263.0
3      47.0
4       6.0
5     328.0
6    1501.0
7      10.0
8       3.0
9      34.0
Name: Reviews, dtype: float64


In [32]:
# From gitbash, you may have to run the following, then restart Jupyter if no map displays:
# $ pip install gmaps
# $ jupyter nbextension enable --py --sys-prefix gmaps
# $ jupyter nbextension enable --py widgetsnbextension

# Create a poverty Heatmap layer
san_diego_coordinates = (32.7839428,-117.2529375)
fig = gmaps.figure(center=san_diego_coordinates, zoom_level=13)
#fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=reviews, 
                                 dissipating=False, max_intensity=1500,
                                 point_radius = .01)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
# Intensity based on maximum value in the 'weights' parameter
heat_layer.max_intensity = 1500
# point_radius is based in km, I think
heat_layer.point_radius = .01

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
#Confirm data has been added by querying the events table
pd.read_sql_query('select * from events', con=engine).head()

,id,Name,Address,Description,Category,Neighborhood,Reviews,Date,Cost
0,1,15th Annual Martin Luther King Jr Day Parade,None,(Not Provided),Events,Downtown,None,2019-01-20,2
1,2,18th Annual St Patricks Day Parade Festival,None,(Not Provided),Events,Balboa Park,None,2019-03-16,1
2,3,3rd Annual San Diego Bird Festival,None,(Not Provided),Events,Mission Bay,None,2019-05-12,2
3,4,4th Annual International Mariachi Festival,None,(Not Provided),Events,South Bay,None,2019-03-10,2
4,5,4th Annual Small Image Show,None,"Art Exhibition Featuring Works 10x10"" and Smaller",Events,Downtown,None,2019-04-21,2


In [34]:
#Confirm data has been added by querying the events table
pd.read_sql_query('select * from attractions', con=engine).head()

,id,Name,Address,Description,Category,Neighborhood,Reviews,Date,Cost
0,1,Action Sport Rentals,"3981 Mission Blvd, San Diego, CA 92109-6958",None,Boat Tours & Water Sports,None,18,None,2
1,2,Adventure Water Sports,"1710 W Mission Bay Dr Dana Hotel, San Diego, C...",None,Boat Tours & Water Sports,None,7,None,2
2,3,Adventure Water Sports,"2211 Pacific Beach Dr, San Diego, CA 92109-5626",None,Boat Tours & Water Sports,None,1,None,2
3,4,Affordable Jet Ski Tours,"4255 Mission Bay Dr, San Diego, CA 92109-5782",None,Boat Tours & Water Sports,None,1,None,2
4,5,Always Summer Surf School,"4904 Crystal Dr, San Diego, CA 92109-2062",None,Boat Tours & Water Sports,None,8,None,2


In [35]:
#Confirm data has been added by querying the events table
pd.read_sql_query('select * from restaurants', con=engine).head()

,id,Name,Address,Description,Category,Neighborhood,Reviews,Date,Cost
0,1,Arslan's Gyros,3861 Mission Blvd,Greek,Food & Drink,None,322,None,2
1,2,BBQ House Bar & Grill,5025 Newport Ave,Bars,Food & Drink,None,23,None,2
2,3,BO-beau kitchen + bar,4996 W Point Loma Blvd,French,Food & Drink,None,6,None,4
3,4,Bao Beach,3735 Mission Blvd,Asian Fusion,Food & Drink,None,36,None,2
4,5,Barefoot Bar & Grill,1404 Vacation Rd,American (New),Food & Drink,None,1526,None,3


In [36]:
#Confirm data has been added by querying the events, attractions and restaurants tables
pd.read_sql_query('select * from events union select * from attractions union select * from restaurants\
    order by Name;', con=engine).head()

,id,Name,Address,Description,Category,Neighborhood,Reviews,Date,Cost
0,1,15th Annual Martin Luther King Jr Day Parade,None,(Not Provided),Events,Downtown,NaN,2019-01-20,2
1,2,18th Annual St Patricks Day Parade Festival,None,(Not Provided),Events,Balboa Park,NaN,2019-03-16,1
2,3,3rd Annual San Diego Bird Festival,None,(Not Provided),Events,Mission Bay,NaN,2019-05-12,2
3,4,4th Annual International Mariachi Festival,None,(Not Provided),Events,South Bay,NaN,2019-03-10,2
4,5,4th Annual Small Image Show,None,"Art Exhibition Featuring Works 10x10"" and Smaller",Events,Downtown,NaN,2019-04-21,2


In [37]:
pd.read_sql_query('select * from (select * from events union select * from attractions union select * from restaurants) as a \
where a.Category = "Food & Drink" order by a.Name;', con=engine).head()

,id,Name,Address,Description,Category,Neighborhood,Reviews,Date,Cost
0,1,Arslan's Gyros,3861 Mission Blvd,Greek,Food & Drink,None,322,None,2
1,11,Baja Winery Tours,"4629 Cass St # 2, San Diego, CA 92109-2805",None,Food & Drink,None,172,None,3
2,4,Bao Beach,3735 Mission Blvd,Asian Fusion,Food & Drink,None,36,None,2
3,5,Barefoot Bar & Grill,1404 Vacation Rd,American (New),Food & Drink,None,1526,None,3
4,6,Bayside Landing,3780 Ingraham St,American (New),Food & Drink,None,236,None,3
